In [1]:
from src.analysis import factorial_anova as fa

2025-03-17 15:15:32,371 - src.logging - INFO - Initializing...
2025-03-17 15:15:32,465 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.


In [2]:
results_path = '../data/math_reasoning/results_20250317_131042_claude_3_5_haiku_20241022_0_3.jsonl'


results = fa.load_data(results_path)

results.head()

,custom_id,correct,template_id,Role_Assignment,Example_Solution,Chain_of_Thought,Self_Verification
0,61a5d380-e8cc-4ebf-a95c-3648416a183c_1353,1,61a5d380-e8cc-4ebf-a95c-3648416a183c,False,False,False,False
1,61a5d380-e8cc-4ebf-a95c-3648416a183c_1353,1,61a5d380-e8cc-4ebf-a95c-3648416a183c,False,False,False,False
2,61a5d380-e8cc-4ebf-a95c-3648416a183c_1353,1,61a5d380-e8cc-4ebf-a95c-3648416a183c,False,False,False,False
3,61a5d380-e8cc-4ebf-a95c-3648416a183c_1353,1,61a5d380-e8cc-4ebf-a95c-3648416a183c,False,False,False,False
4,61a5d380-e8cc-4ebf-a95c-3648416a183c_1353,1,61a5d380-e8cc-4ebf-a95c-3648416a183c,False,False,False,False


In [3]:
factor_columns = [
    'Role_Assignment',
    'Chain_of_Thought',
    # 'Tree_of_Thoughts',
    'Self_Verification',
    'Example_Solution',
]


metric_column = 'correct'

all_columns = factor_columns + [metric_column]

In [4]:
for column in factor_columns:
    results[column] = results[column].astype(int)
results.head()

,custom_id,correct,template_id,Role_Assignment,Example_Solution,Chain_of_Thought,Self_Verification
0,61a5d380-e8cc-4ebf-a95c-3648416a183c_1353,1,61a5d380-e8cc-4ebf-a95c-3648416a183c,0,0,0,0
1,61a5d380-e8cc-4ebf-a95c-3648416a183c_1353,1,61a5d380-e8cc-4ebf-a95c-3648416a183c,0,0,0,0
2,61a5d380-e8cc-4ebf-a95c-3648416a183c_1353,1,61a5d380-e8cc-4ebf-a95c-3648416a183c,0,0,0,0
3,61a5d380-e8cc-4ebf-a95c-3648416a183c_1353,1,61a5d380-e8cc-4ebf-a95c-3648416a183c,0,0,0,0
4,61a5d380-e8cc-4ebf-a95c-3648416a183c_1353,1,61a5d380-e8cc-4ebf-a95c-3648416a183c,0,0,0,0


In [5]:
formula = fa.get_formula(metric_column, factor_columns)

print(formula)

correct ~ Role_Assignment * Chain_of_Thought * Self_Verification * Example_Solution


In [6]:
data = results[all_columns].copy().astype(int)
data.head()

,Role_Assignment,Chain_of_Thought,Self_Verification,Example_Solution,correct
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


In [7]:
for column in factor_columns:
    print(data.groupby(column)['correct'].mean())


Role_Assignment
0    0.5225
1    0.5550
Name: correct, dtype: float64
Chain_of_Thought
0    0.5450
1    0.5325
Name: correct, dtype: float64
Self_Verification
0    0.5350
1    0.5425
Name: correct, dtype: float64
Example_Solution
0    0.5600
1    0.5175
Name: correct, dtype: float64


In [8]:
model = fa.get_fitted_model(data, formula)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                correct   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     21.88
Date:                Mon, 17 Mar 2025   Prob (F-statistic):           1.36e-60
Time:                        15:15:43   Log-Likelihood:                -28748.
No. Observations:               40000   AIC:                         5.753e+04
Df Residuals:                   39984   BIC:                         5.766e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================================
                                                                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                               0.5600      0.010     56.388      0.000       0.541       0.579
Role_Assignment                                                        -0.0400      0.014     -2.848      0.004      -0.068      -0.012
Chain_of_Thought                                                       -0.0400      0.014     -2.848      0.004      -0.068      -0.012
Role_Assignment:Chain_of_Thought                                        0.1600      0.020      8.055      0.000       0.121       0.199
Self_Verification                                                       0.0400      0.014      2.848      0.004       0.012       0.068
Role_Assignment:Self_Verification                                       0.0400      0.020      2.014      0.044       0.001       0.079
Chain_of_Thought:Self_Verification                                     -0.0600      0.020     -3.021      0.003      -0.099      -0.021
Role_Assignment:Chain_of_Thought:Self_Verification                     -0.1200      0.028     -4.272      0.000      -0.175      -0.065
Example_Solution                                                       -0.0400      0.014     -2.848      0.004      -0.068      -0.012
Role_Assignment:Example_Solution                                        0.0200      0.020      1.007      0.314      -0.019       0.059
Chain_of_Thought:Example_Solution                                       0.0200      0.020      1.007      0.314      -0.019       0.059
Role_Assignment:Chain_of_Thought:Example_Solution                      -0.1200      0.028     -4.272      0.000      -0.175      -0.065
Self_Verification:Example_Solution                                     -0.0400      0.020     -2.014      0.044      -0.079      -0.001
Role_Assignment:Self_Verification:Example_Solution                  -3.073e-16      0.028  -1.09e-14      1.000      -0.055       0.055
Chain_of_Thought:Self_Verification:Example_Solution                     0.0200      0.028      0.712      0.476      -0.035       0.075
Role_Assignment:Chain_of_Thought:Self_Verification:Example_Solution     0.1800      0.040      4.531      0.000       0.102       0.258
==============================================================================
Omnibus:                   141244.864   Durbin-Watson:                   0.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6455.025
Skew:                          -0.153   Prob(JB):                         0.00
Kurtosis:                       1.056   Cond. No.                         47.0
==============================================================================

Notes:
[1] Standard Erro

### Interpretation

Looking at these regression results:

1. The model is statistically significant (F=21.88, p<0.001) but explains only a small portion of variance (R²=0.008).
2. Key findings:
    - Individually, Role_Assignment, Chain_of_Thought, and Example_Solution each reduce correctness by 4% (all p=0.004)
    - Self_Verification individually increases correctness by 4% (p=0.004)
    - The Role_Assignment + Chain_of_Thought interaction is strongly positive (16%, p<0.001)
    - The 4-way interaction shows a substantial positive effect (18%, p<0.001)
3. Notable interaction patterns:
    - Chain_of_Thought works better with Role_Assignment than alone
    - Self_Verification has negative interactions with Chain_of_Thought
    - Three-way interactions show complex patterns

In [9]:
type_III_anova_table = fa.run_factorial_anova_from_model(model, anova_type='III')
type_III_anova_table

,sum_sq,df,F,PR(>F)
Intercept,7.840000e+02,1.0,3.179578e+03,0.000000e+00
Role_Assignment,2.000000e+00,1.0,8.111167e+00,4.401582e-03
Chain_of_Thought,2.000000e+00,1.0,8.111167e+00,4.401582e-03
Role_Assignment:Chain_of_Thought,1.600000e+01,1.0,6.488934e+01,8.140116e-16
Self_Verification,2.000000e+00,1.0,8.111167e+00,4.401582e-03
Role_Assignment:Self_Verification,1.000000e+00,1.0,4.055584e+00,4.403218e-02
Chain_of_Thought:Self_Verification,2.250000e+00,1.0,9.125063e+00,2.522904e-03
Role_Assignment:Chain_of_Thought:Self_Verification,4.500000e+00,1.0,1.825013e+01,1.941602e-05
Example_Solution,2.000000e+00,1.0,8.111167e+00,4.401582e-03
Role_Assignment:Example_Solution,2.500000e-01,1.0,1.013896e+00,3.139774e-01


### Type-III ANOVA Interpretation

1. All main effects are significant (p < 0.005):
    - Each individual prompt component has a statistically significant effect on correctness
2. Most significant interactions:
    - Role_Assignment × Chain_of_Thought (F=64.89, p<0.001) - extremely strong interaction
    - 4-way interaction (F=20.53, p<0.001) - complex interplay between all components
    - Role_Assignment × Chain_of_Thought × Self_Verification (F=18.25, p<0.001)
    - Role_Assignment × Chain_of_Thought × Example_Solution (F=18.25, p<0.001)

The strong interactions confirm that certain prompt components work synergistically while some combinations may be counterproductive